### Desarrollado por:
- Jorge Gutiérrez Mata
- Luis Adrián Badilla Godínez

### Topic Summary
- Transformación de Gauss con Scikit-learn.
- Transformación de Gauss con Feature-Engine.
- Discretización
- Discretización de igual ancho.
- Discretización de igual frecuencia.

Para el desarrollo de este cuaderno se irán cubriendo los temas mencionados anteriormente. Se divide primero en teoría seguido de un ejemplo de utilización con datasets definidos. 

Los link de descarga para los dataset son los siguientes:
1. abc: 
1. cde: 
1. ghi: 
1. jkl: 
 
### Transformación de Gauss con Scikit-learn.
### Transformación de Gauss con Feature-Engine.

### Discretización
Este es un proceso que consiste de transformar variables continuas en variables discretas por medio de la creación de intervalos continuos que abarquen el rango de valores de las variables discretas.

Este proceso ayuda a manejar datos atípicos colocando estos datos en los intervalos más bajos o altos. Por lo cual estas observaciones atípicas ya no difieren con el resto de los valores en las colas de distribución. 

Además de esto ayuda con las variables sesgadas al distribuirlas en conjuntos de intervalos con el mismo número de observaciones.

### D

### Discretización de igual ancho.
### Discretización de igual frecuencia.